# Vector Store

In [ ]:
from devtools import debug

!export PYTHONPATH=":./python"

### Split the text into chunks

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from python.ai_core.embeddings import EmbeddingsFactory
from python.ai_core.vector_store import VectorStoreFactory

loader = TextLoader("use_case_data/other/state_of_the_union.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
debug(texts)

### Calculate embeddings and oyt them in a vector database .

This is a adaptation of https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/ <br><br>
We have a factory to facilitate Vector Store configuration and selection. <br>
One benefit if that it's easy to change vector store type and embedding model, just by configuration.

In [13]:
vs_factory = VectorStoreFactory(
    id="Chroma_in_memory",
    collection_name="maintenance_procedure",
    embeddings_factory=EmbeddingsFactory(),
)

debug(vs_factory)

db = vs_factory.vector_store
db.add_documents(texts)

2024-06-26 17:48:30.597 | INFO     | python.ai_core.vector_store:vector_store:113 - get vector store  : Chroma_in_memory/maintenance_procedure_ada_002_edenai


/tmp/ipykernel_312843/4177014484.py:10 <module>
    vs_factory: VectorStoreFactory(
        id='Chroma_in_memory',
        embeddings_factory=EmbeddingsFactory(
            embeddings_id='ada_002_edenai',
            encoding_str=None,
            retrieving_str=None,
            info=EMBEDDINGS_INFO(
                id='ada_002_edenai',
                cls='EdenAiEmbeddings',
                model='openai/1536__text-embedding-ada-002',
                key='EDENAI_API_KEY',
                prefix='',
            ),
        ),
        collection_name='maintenance_procedure',
        index_document=False,
        collection_metadata=None,
        collection_full_name='maintenance_procedure_ada_002_edenai',
        description='Chroma_in_memory/maintenance_procedure_ada_002_edenai',
        vector_store=<langchain_community.vectorstores.chroma.Chroma object at 0x7f65859f57e0>,
    ) (VectorStoreFactory)


### Test a query

In [16]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query, k=3)
debug(docs)

/tmp/ipykernel_312843/3766655050.py:3 <module>
    docs: [
        Document(
            page_content=(
                'Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act'
                '. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n'
                '\n'
                'Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen '
                'Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Cou'
                'rt. Justice Breyer, thank you for your service. \n'
                '\n'
                'One of the most serious constitutional responsibilities a President has is nominating someone to serv'
                'e on the United States Supreme Court. \n'
                '\n'
                'And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brow

[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': 'use_case_data/other/state_of_the_union.txt'}),
 Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voti

### Vector Store as Runnable

In [15]:
retriever = db.as_retriever()

retriever.invoke(query)

[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': 'use_case_data/other/state_of_the_union.txt'}),
 Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voti

### Configurable Retriever Runnable 

It's a little bit tricky, so there is a small wrapper in the Factory: 

In [20]:
retriever = vs_factory.get_configurable_retriever(default_k=20)

config = {"configurable": {"search_kwargs": {"k": 1}}}
result = retriever.invoke(query, config=config)
result  # return 1 result

[Document(page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.', metadata={'source': 'use_case_data/other/state_of_the_union.txt'})]